In [2]:
import pandas as pd

Загружаем данные, производим оптимизацию.

In [3]:

train = pd.read_csv('train.csv', sep=',', index_col=0,
                       dtype = {'row_id': 'int64', 'timestamp': 'int64', 'user_id': 'int32', 'content_id': 'int16', 
                                'content_type_id': 'int8', 'task_container_id': 'int16', 'user_answer': 'int8', 
                                'answered_correctly': 'int8', 'prior_question_elapsed_time': 'float32', 
                                'prior_question_had_explanation': 'boolean'
                               }
                      )

In [4]:
train.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
row_id,,,,,,,,,
0,0,115,5692,0,1,3,1,NaN,<NA>
1,56943,115,5716,0,2,2,1,37000.0,False
2,118363,115,128,0,0,0,1,55000.0,False
3,131167,115,7860,0,3,0,1,19000.0,False
4,137965,115,7922,0,4,1,1,11000.0,False


In [5]:

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101230332 entries, 0 to 101230331
Data columns (total 9 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   timestamp                       int64  
 1   user_id                         int32  
 2   content_id                      int16  
 3   content_type_id                 int8   
 4   task_container_id               int16  
 5   user_answer                     int8   
 6   answered_correctly              int8   
 7   prior_question_elapsed_time     float32
 8   prior_question_had_explanation  boolean
dtypes: boolean(1), float32(1), int16(2), int32(1), int64(1), int8(3)
memory usage: 3.1 GB


Переводим данные к числовому типу, заполняем пустоты

In [6]:
for i in range(train.shape[0]):
    if pd.isna(train['prior_question_had_explanation'][i]) == True:
        train['prior_question_had_explanation'][i] = False
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype('int8')

Смотрим результат

In [7]:
train.head()

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
row_id,,,,,,,,,
0,0,115,5692,0,1,3,1,NaN,0
1,56943,115,5716,0,2,2,1,37000.0,0
2,118363,115,128,0,0,0,1,55000.0,0
3,131167,115,7860,0,3,0,1,19000.0,0
4,137965,115,7922,0,4,1,1,11000.0,0


In [8]:
unique_list = []
for col in train.columns:
    item = (col, train[col].nunique(), train[col].dtype)
    unique_list.append(item)
unique_counts = pd.DataFrame(unique_list,
                             columns=['Column_Name', 'Num_Unique', 'Type']
                            ).sort_values(by='Num_Unique',  ignore_index=True)
display(unique_counts)

,Column_Name,Num_Unique,Type
0,content_type_id,2,int8
1,prior_question_had_explanation,2,int8
2,answered_correctly,3,int8
3,user_answer,5,int8
4,prior_question_elapsed_time,3258,float32
5,task_container_id,10000,int16
6,content_id,13782,int16
7,user_id,393656,int32
8,timestamp,72821015,int64


In [20]:
train['content_type_id'].value_counts(normalize=True) 
train['content_type_id'].value_counts()[0] 
train['content_type_id'].value_counts()[1] 
train['content_type_id'].value_counts()[0] / train['content_type_id'].value_counts()[1]
train['content_type_id'].mean() * 100 , '%' 

(1.9352223402764304, '%')

Отношение вопросов к лекциям, примерно 50 вопросов на одну лекцию, 1,98%

In [18]:
train['prior_question_elapsed_time'].mean() / 1000

25.42384375

25,42 секунды среднее время ответа на вопрос

In [21]:
print(train[train['content_type_id'] == 0].shape[0] / train['user_id'].nunique(),
train[train['answered_correctly'] != -1]['answered_correctly'].sum() / train['user_id'].nunique(),
train[train['content_type_id'] == 1].shape[0] / train['user_id'].nunique(),
train['prior_question_had_explanation'].sum() / train['user_id'].nunique(),
train['prior_question_elapsed_time'].mean())

252.17778974536142 165.74020718596947 4.976507407482675 227.82724002682545 25423.844


Cреднее число отвеченных вопросов: 252.17
Среднее кол-во правильных ответов: 165.74
Среднее число просмотренных лекций: 4.97
Среднее число виденных объяснений: 227.82